0. Imports

In [30]:
import pandas as pd
import numpy as np

1. Load csv

In [31]:
# change to your file location
df = pd.read_csv('/content/drive/MyDrive/Škola/DM/spravanie_studentov_v_zavislosti_od_casu_IV/MLM_STUD_stats.csv', ';', usecols=range(0,10))
df.head(10)

,Effect,Intercept,HODINA,HODINA_STV,PO,UT,STR,STVR,PIA,SO
0,uvod_STUD,1.34835,0.34182,-0.01563,-0.63353,1.10307,0.11376,-0.49911,0.68682,0
1,studium_STUD,-2.84924,0.38448,-0.00867,0.95518,3.02822,1.54827,1.22148,1.20767,0
2,oznamy_STUD,1.51459,-0.38342,0.01248,1.65098,3.04913,1.59056,0.14253,-1.24192,0


2. Create collection of weekdays

In [32]:
days = ['PO', 'UT', 'STR', 'STVR', 'PIA', 'SO']

3. Create estimates for web parts

In [33]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
index = 0

# Cycle through hours from 7 to 23
for x in range (7,23):
      new_row_uvod = {}
      new_row_studium = {}
      new_row_oznamy = {}
      i = 1

      # Cycle through weekdays
      for day in days:

        # Create logits estimates
        logit_uvod = df.at[index, 'Intercept'] + df.at[index, 'HODINA']*x+df.at[index, 'HODINA_STV']*(x*x)+df.at[index, day]
        logit_studium = df.at[index+1, 'Intercept'] + df.at[index+1, 'HODINA']*x+df.at[index+1, 'HODINA_STV']*(x*x)+df.at[index+1, day]
        logit_oznamy = df.at[index+2, 'Intercept'] + df.at[index+2, 'HODINA']*x+df.at[index+2, 'HODINA_STV']*(x*x)+df.at[index+2, day]
        
        reference_web = 1 / (1 + np.exp(logit_uvod) + np.exp(logit_studium) + np.exp(logit_oznamy))
        
        # Create estimates for web parts
        estimate_uvod = np.exp(logit_uvod) * reference_web
        estimate_studium = np.exp(logit_studium) * reference_web
        estimate_oznamy = np.exp(logit_oznamy) * reference_web

        den = str(i) + '_' + day

        # Create new rows and append it to dataframe
        new_row_uvod.update({den: estimate_uvod})
        new_row_studium.update({den: estimate_studium})
        new_row_oznamy.update({den: estimate_oznamy})
        i = i + 1
      
      # Append time to rows
      new_row_uvod.update({'0_hod': x})
      new_row_studium.update({'0_hod': x})
      new_row_oznamy.update({'0_hod': x})

      # Update dataframes
      df1 = df1.append(new_row_uvod, sort=False, ignore_index=True)
      df2 = df2.append(new_row_studium, sort=False, ignore_index=True)
      df3 = df3.append(new_row_oznamy, sort=False, ignore_index=True)

df1.head()

,0_hod,1_PO,2_UT,3_STR,4_STVR,5_PIA,6_SO
0,7.0,0.701022,0.714323,0.801568,0.823223,0.950371,0.945436
1,8.0,0.728082,0.726046,0.811025,0.816037,0.944546,0.948246
2,9.0,0.737846,0.722664,0.808724,0.800848,0.936567,0.947312
3,10.0,0.733773,0.707178,0.796942,0.778498,0.926114,0.943307
4,11.0,0.718087,0.681387,0.776758,0.749159,0.912671,0.936379


4. Export to excel

In [34]:
# Creating Excel Writer Object from Pandas  
writer = pd.ExcelWriter('STUD_accessing_the_web_parts.xlsx',engine='xlsxwriter')   
workbook=writer.book
worksheet=workbook.add_worksheet('STUD')
writer.sheets['STUD'] = worksheet

# Úvod
worksheet.write(0, 0, "Úvod")
df1.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=0, index=False)

# Śtúdium
worksheet.write(0, 8, "Štúdium")
df2.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=8, index=False)

# Oznamy
worksheet.write(0, 16, "Oznamy")
df3.to_excel(writer, sheet_name='STUD',startrow=1 , startcol=16, index=False)
writer.save()